In [ ]:
import secml_malware
from secml.array import CArray
import os
import magic
import numpy as np
import torch
import configparser
from secml_malware.attack.blackbox.c_wrapper_phi import CEnd2EndWrapperPhi
from secml_malware.attack.blackbox.ga.c_base_genetic_engine import CGeneticAlgorithm
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware
from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel
from secml_malware.attack.blackbox.c_gamma_sections_evasion import CGammaSectionsEvasionProblem
from secml_malware.attack.whitebox.c_header_evasion import CHeaderEvasion
from secml_malware.attack.whitebox.c_extend_dos_evasion import CExtendDOSEvasion
from secml_malware.attack.whitebox.c_shift_evasion import CContentShiftingEvasion
from secml_malware.attack.whitebox import CKreukEvasion
net = MalConv()
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
section = 'ad_sample'
attack_type = config.getint(section, 'attack_type')
goodware_files = config.get(section, 'goodware_files')
predicted_malware = config.get(section, 'pr_malware')
full_dos_f = config.get(section, 'full_dos_f')
extend_dos_f = config.get(section, 'extend_dos_f')
c_shift_f = config.get(section, 'c_shift_f')
fgsm_f = config.get(section, 'fgsm_f')
gamma_f = config.get(section, 'gamma_f')
c_shift_extension = config.getint(section,'c_shift_extension')
fgsm_padding_bytes = config.getint(section, 'fgsm_padding_bytes')
ep_value = config.getfloat(section, 'epsilon_value')
thresh = config.getfloat(section, 'threshold')
all_dos = config.getboolean(section, 'all_dos')
header_extension = config.getint(section, 'header_extension')
gamma_sections = config.getint(section, 'gamma_sections')
population = config.getint(section, 'population_size')
gamma_p_regularizer = config.getfloat(section, 'gamma_p_regularizer')
gamma_iterations = config.getint(section, 'gamma_iterations')

In [ ]:
X = []
y = []
f_name = []
file_path = []
for i, f in enumerate(os.listdir(predicted_malware)):
    path = os.path.join(predicted_malware, f)
    if "PE32" not in magic.from_file(path):
        print("pe32 not in path")
        continue
    
    with open(path, "rb") as file_handle:
        code = file_handle.read()
        
        
    x = End2EndModel.bytes_to_numpy(
        code, net.get_input_max_length(),256, False)
    _, confidence = net.predict(CArray(x), True)
    if confidence[0, 1].item() < 0.5:
        continue
    X.append(x)
    conf = confidence[1][0].item()
    y.append([1 - conf, conf])
    file_path.append(path)
    f_name.append(f)
    
    

In [ ]:
if attack_type ==1:
    full_dos = CHeaderEvasion(net, optimize_all_dos=all_dos, threshold= thresh)
    i = 0
    for sample, label in zip(X, y):
        y_pred, adv_score, adv_ds, f_obj = full_dos.run(CArray(sample), CArray(label[1]))
        adv_x = adv_ds.X[0,:]
        real_adv_x = full_dos.create_real_sample_from_adv(file_path[i], adv_x,os.path.join(full_dos_f,str(f_name[i])))
        real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
        _, confidence = net.predict(CArray(real_x), True)
        if confidence[1]>confidence[0]:
            os.remove(os.path.join(full_dos_f,str(f_name[i])))
        i=i+1
       

if attack_type ==2:
    extend_dos = CExtendDOSEvasion(net, pe_header_extension = header_extension, threshold= thresh)
    i = 0
    for sample, label in zip(X, y):
        y_pred, adv_score, adv_ds, f_obj = extend_dos.run(CArray(sample), CArray(label[1]))
        adv_x = adv_ds.X[0,:]
        real_adv_x = extend_dos.create_real_sample_from_adv(file_path[i], adv_x,os.path.join(extend_dos_f,str(f_name[i])))
        real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
        _, confidence = net.predict(CArray(real_x), True)
        if confidence[1]>confidence[0]:
            os.remove(os.path.join(extend_dos_f,str(f_name[i])))
        i=i+1
        
        
if attack_type ==3:
    content_shift = CContentShiftingEvasion(net,preferable_extension_amount=c_shift_extension, threshold= thresh)
    i = 0
    for sample, label in zip(X, y):
        y_pred, adv_score, adv_ds, f_obj = content_shift.run(CArray(sample), CArray(label[1]))
        adv_x = adv_ds.X[0,:]
        real_adv_x = content_shift.create_real_sample_from_adv(file_path[i], adv_x,os.path.join(c_shift_f,str(f_name[i])))
        real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
        _, confidence = net.predict(CArray(real_x), True)
        if confidence[1]>confidence[0]:
            os.remove(os.path.join(c_shift_f,str(f_name[i])))
        i=i+1
        
        
if attack_type ==4:
    fgs = CKreukEvasion(net, how_many_padding_bytes=fgsm_padding_bytes, epsilon=ep_value, threshold = thresh)
    i = 0
    for sample, label in zip(X, y):
        y_pred, adv_score, adv_ds, f_obj = fgs.run(CArray(sample), CArray(label[1]))
        adv_x = adv_ds.X[0,:]
        real_adv_x = fgsm.create_real_sample_from_adv(file_path[i], adv_x,os.path.join(fgsm_f,str(f_name[i])))
        real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
        _, confidence = net.predict(CArray(real_x), True)
        if confidence[1]>confidence[0]:
            os.remove(os.path.join(fgsm_f,str(f_name[i])))
        i=i+1
        
        
if attack_type ==5:
    net = CEnd2EndWrapperPhi(net)
    X = []
    y = []
    file_path = []
    f_name = []
    for i, f in enumerate(os.listdir(predicted_malware)):
        path = os.path.join(predicted_malware, f)
        if "PE32" not in magic.from_file(path):
            os.remove(path)
            continue
        with open(path, "rb") as file_handle:
            code = file_handle.read()
        x = CArray(np.frombuffer(code, dtype=np.uint8)).atleast_2d()
        _, confidence = net.predict(x, True)
        
        if confidence[0, 1].item() < 0.5:
            continue
        X.append(x)
        conf = confidence[1][0].item()
        y.append([1 - conf, conf])
        file_path.append(path)
        f_name.append(f)
    section_population, what_from_who = CGammaSectionsEvasionProblem.create_section_population_from_folder(goodware_files, how_many=gamma_sections, sections_to_extract=['.rdata'])
    attack = CGammaSectionsEvasionProblem(section_population, net, population_size=population, penalty_regularizer=gamma_p_regularizer, iterations = gamma_iterations)
    engine = CGeneticAlgorithm(attack)
    i = 0
    for sample, label in zip(X, y):
        y_pred, adv_score, adv_ds, f_obj = engine.run(sample, CArray(label[1]))
        adv_x = adv_ds.X[0, :]
        engine.write_adv_to_file(adv_x, os.path.join(gamma_f,str(f_name[i])))
        with open(os.path.join(gamma_f,str(f_name[i])), 'rb') as h:
            code = h.read()
        real_adv_x = CArray(np.frombuffer(code, dtype=np.uint8))
        _, confidence = net.predict(CArray(real_adv_x), True)
        if confidence[1]>confidence[0]:
            os.remove(os.path.join(gamma_f,str(f_name[i])))
        i=i+1
        
   
    
